# Implementation of a Newton–Raphson solver for nonlinear elasticity

In this notebook, we will analyse the same problem as in the previous notebook [HyperelasticSolid.ipynb](HyperelasticSolid.ipynb). However, this time, we will implement or own Newton–Raphson solver, rather than rely on FEniCS's `NewtonSolver`. 

This notebook is inspired by the demo: https://jorgensd.github.io/dolfinx-tutorial/chapter4/newton-solver.html.


You need to fill in the blank lines that are indicated as follows
```
# COMPLETE THIS LINE or XXX
```

Until you do that, the Notebook will *not* work properly.

Please complete the code and answear questions (these questions here and there in the code are typical questions that we could ask you during the final oral exam).

## Setting up the problem

The code below is merely a copy/paste of the code from the previous sessions, as we use the same problem to illustrate the implementation of the Newton–Raphson iterations

We first import and setup the usual modules.

In [ ]:
import dolfinx 
from dolfinx import nls
import matplotlib.pyplot as plt
import ufl 
import numpy as np
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
from petsc4py import PETSc
import pyvista
import extract
from dolfinx import log

### Parameters of the simulation

Place here the parameters that can be changed without altering the logics of the code.

In [ ]:
Lx, Ly = 1.0, 0.1 # Dimensions of the beam
nx, ny = 60, 6 # Number of elements in each direction
Y, nu = 1, 0.3 # Young modulus and Poisson ratio
rhog_light = 0.005 # small weight
print('Gamma light',12*rhog_light*Lx**3/Ly**2)  # Gamma = Mg / (EI/L^2)
rhog_heavy = 0.01 # large weight
print('Gamma heavy',12*rhog_heavy*Lx**3/Ly**2)  # Gamma = Mg / (EI/L^2)
output_dir = "nr_output"

### Mesh

In [ ]:
my_domain = dolfinx.mesh.create_rectangle(comm=MPI.COMM_WORLD, points=((0,0),(Lx,Ly)), n=(nx, ny), 
                                     cell_type=dolfinx.mesh.CellType.triangle)

def left(x):
    return np.isclose(x[0], 0)

def right(x):
    return np.isclose(x[0], Lx)

fdim = my_domain.topology.dim -1
left_facets = dolfinx.mesh.locate_entities_boundary(my_domain, fdim, left)
right_facets = dolfinx.mesh.locate_entities_boundary(my_domain, fdim, right)

# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with 2
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = dolfinx.mesh.meshtags(my_domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

ds = ufl.Measure('ds', domain=my_domain, subdomain_data=facet_tag)
dx = ufl.Measure("dx", domain=my_domain)

### Function space

In [ ]:
V = dolfinx.fem.VectorFunctionSpace(my_domain, ("CG", 2))
u = dolfinx.fem.Function(V)

### Potential energy

We first define the strain energy of a (compressible) Kirchhoff Saint-Venant material.

In [ ]:
dim = len(u)
I = ufl.variable(ufl.Identity(dim))
F = ufl.variable(I + ufl.grad(u))
C = ufl.variable(F.T * F)
E = ufl.variable(1/2*(C-I))
Ic = ufl.tr(C)
Jdet  = ufl.det(F)
mu = Y/(2*(1 + nu))
lmbda = Y*nu/((1 + nu)*(1 - 2*nu))
# lmbda2D = 2*lmbda*mu/(lmbda + 2*mu) 
psi = lmbda*ufl.tr(E)**2/2+mu*ufl.inner(E,E) # kirchhoff saint venant 
#psi = (mu/2)*(Ic - 2) - mu*ufl.ln(Jdet) + (lmbda/2)*(ufl.ln(Jdet))**2 # Neohookean

We then define the loading (body forces $b_0 = \rho \, g$).

In [ ]:
rhog = dolfinx.fem.Constant(my_domain, ScalarType((0, 0)))

We can now define the potential energy and its derivatives w.r.t the displacement ```u```.

In [ ]:
potential_energy = psi*dx - ufl.inner(u, rhog)*dx
v = ufl.TestFunction(V)
residual = ufl.derivative(potential_energy, u, v)

Boundary conditions for a built-in support on the left-hand side.

In [ ]:
left_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
u_bc = np.array((0,) * my_domain.geometry.dim, dtype=ScalarType)
bc = dolfinx.fem.dirichletbc(u_bc, left_dofs,V)

### Computing the reference solution

We use the built-in non-linear solver to compute a reference solution, which we will compare to the solution computed with our own NR solver.

In [ ]:
# We use dolfinx newton solver to compute the reference solution
log.set_log_level(log.LogLevel.INFO)
problem = dolfinx.fem.petsc.NonlinearProblem(residual, u, bcs=[bc])
solverNL = nls.petsc.NewtonSolver(my_domain.comm, problem)
solverNL.atol = 1e-8
solverNL.rtol = 1e-8
solverNL.max_it = 120
solverNL.convergence_criterion = "incremental" 
u.x.set(0) # the seed sent to newton is the undeformed state u(x,y)=0 for all x,y
with dolfinx.io.XDMFFile(my_domain.comm, output_dir+"/ref_solution.xdmf", "w") as file:
    file.write_mesh(my_domain) # we export the mesh to paraview
    
rhog.value[1] = -rhog_light # we set the intensity of gravity
num_its, converged = solverNL.solve(u) # we call the dolfinx newton solver
print('number of iterations in Newton solver:',num_its)
print('Has it converged?:',converged)
with dolfinx.io.XDMFFile(my_domain.comm, output_dir+"/ref_solution.xdmf", "a") as file:
    file.write_function(u) # we export the deformed configuration to paraview
displ = dolfinx.fem.assemble_scalar(dolfinx.fem.form(u[1]*ds(2)))/Ly # we compute < u_y(L,y) >
print('End-displacement = ',displ)
log.set_log_level(log.LogLevel.ERROR)

In [ ]:
u_ref = u.copy() # we keep this reference solution 

## Implementation of the Newton–Raphson solver

A few definitions.

In [ ]:
J = ufl.derivative(XXX, u) # jacobian of the newton method
J_form = dolfinx.fem.form(J)
r_form = dolfinx.fem.form(residual)
A = dolfinx.fem.petsc.create_matrix(J_form)
L = dolfinx.fem.petsc.create_vector(r_form)
# Which system is solved at each Newton iteration?
# What is the unknown?

Setting up the linear solver, you can try to change the options

In [ ]:
solver = PETSc.KSP().create(my_domain.comm) # linear solver in our newton method
opts = PETSc.Options()
opts["ksp_type"] = "cg" # choices "preonly" "cg"
opts["ksp_rtol"] = 1.0e-10
opts["ksp_max_it"] = 500
opts["pc_type"] = "lu" # (precond) choices : "gamg" "lu" "none" "mg" "hypre"
solver.setFromOptions()

In [ ]:
solver.setOperators(XXX) # we should put the matrix operator here
du = dolfinx.fem.Function(V) # this is the unkown of the linear solver du = u_{k+1} - u_k

In [ ]:
i = 0
rhog.value[1] = - rhog_light # we set the intensity of gravity
max_iterations = 35
u.x.set(0) # the seed sent to newton is the undeformed state u(x,y)=0 for all x,y
liste_correction_norm = []
liste_residual_norm = []
liste_L2_err_norm = []
while i < max_iterations:
    # At each step, we solve a linear problem
    
    # Assemble Jacobian and residual
    # Question: Why do we have to re-evaluate the jacobian and residual
    # at _each_ step ?
    
    XXX
    
    XXX
    
    # Scale residual by -1
    L.scale(-1) # Question: why do we do this?
    
    # for dirichelet boudary conditions, we want ot work with symetrical matrices
    dolfinx.fem.petsc.apply_lifting(XXX)
    dolfinx.fem.petsc.set_bc(XXX)

    
    # Solve linear problem
    solver.solve(XXX, du.vector)
    
    # update the displacement field u
    u.x.array[:] += XXX
    i = i + 1

    
    # Compute norm of update
    correction_norm = XXX
    liste_correction_norm.append(correction_norm)
    residu_norm = np.linalg.norm(L.array)
    liste_residual_norm.append(residu_norm)
    L2_error_norm = XXX
    liste_L2_err_norm.append(XXX)
    print(f"Iteration {i}: Correction norm {correction_norm:3.2e}, Residual norm {residu_norm:3.2e}, Norm_L2(u_ref-u) {L2_error_norm:3.2e}")
    
    if L2_error_norm < 1e-14:
        break
    if correction_norm < 1e-10:
        break

In [ ]:
displ = dolfinx.fem.assemble_scalar(dolfinx.fem.form(u[1]*ds(2)))/Ly # we compute < u_y(L,y) >
print('End-displacement = ',displ)
# is it the same value as before?

In [ ]:
# Please plot u(x,0) for the reference solution and the
# solution you have juste found

In [ ]:
# Please compute the L2 error norm 

In [ ]:
# Please export the deformed shape in paraview and compare with the reference solution

In [ ]:
# Please plot the norm of du, the norm of the residual, and the L2 error as function
# of the iteration 

In [ ]:
# Please plot the L2 error at step k+1 as function of the L2 error at step k
# Is it ordre 1, 2, or 3?

In [15]:
# Export to xdmf format the different configurations at each newton iteration
# Are these configurations "equilibrium configuration"?

# Rotation of the gravity field

In [ ]:
# Change the orientation of gravity, and compute u_y(L,0) for alpha_i in [-pi/2;pi/2]
rhog.value[0]=-rhog_light*np.cos(alpha_i)
rhog.value[1]=-rhog_light*np.sin(alpha_i)
# do the same for rhog_heavy

`Question` : how many solutions are there for each value of alpha_i?

# `Further questions`

`Question 1` Sometimes the Jacobian is too long to compute. Only compute it once (or twice, or ...) and keep it the same for the remaining of the iterations and see if your obtain convergence.

`Question 2` Refine the mesh and observe how the number of steps to obtained convergence is affected.
